In [ ]:
import os
import time
import threading
from google.colab import drive

print("⚙️ Installing dependencies...")
!pip install python-chess zstandard -q

import chess.pgn
import zstandard as zstd
import io

print("\n🔗 Mounting Google Drive...")
drive.mount('/content/drive')

# Project structure
PROJECT_ROOT = '/content/drive/MyDrive/GambitFlow_Project/Synapse_Data'
DATA_FACTORY = os.path.join(PROJECT_ROOT, '2')
os.makedirs(DATA_FACTORY, exist_ok=True)

print(f"✅ Workspace: {DATA_FACTORY}")

# Keep-alive function
def keep_alive():
    while True:
        time.sleep(60)

threading.Thread(target=keep_alive, daemon=True).start()
print("✅ Keep-alive active")

⚙️ Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 55.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done

🔗 Mounting Google Drive...
Mounted at /content/drive
✅ Workspace: /content/drive/MyDrive/GambitFlow_Project/Synapse_Data/2
✅ Keep-alive active


In [ ]:
# ==============================================================================
# 📥 DOWNLOAD MULTIPLE DATABASES (Lichess Elite - High Quality)
# ==============================================================================

import requests
import shutil
import os
import zipfile

# 1. Configuration: List of URLs to download
# (আপনি এখানে যত খুশি লিংক অ্যাড করতে পারেন)
DOWNLOAD_URLS = [
    "https://database.nikonoel.fr/lichess_elite_2025-07.zip",
    "https://database.nikonoel.fr/lichess_elite_2025-06.zip",
    "https://database.nikonoel.fr/lichess_elite_2025-05.zip",
    "https://database.nikonoel.fr/lichess_elite_2025-04.zip",
]

# Paths
LOCAL_DIR = "/content/data"
DRIVE_DIR = os.path.join(DATA_FACTORY, "raw_pgns") # ড্রাইভের ভেতর আলাদা ফোল্ডার

os.makedirs(LOCAL_DIR, exist_ok=True)
os.makedirs(DRIVE_DIR, exist_ok=True)

print(f"🎯 Targets: {len(DOWNLOAD_URLS)} files")
print(f"📂 Local Dir: {LOCAL_DIR}")
print(f"💾 Drive Backup: {DRIVE_DIR}\n")

def download_and_extract(url):
    filename = url.split('/')[-1]
    local_path = os.path.join(LOCAL_DIR, filename)
    drive_path = os.path.join(DRIVE_DIR, filename)

    # A. Check if already processed (extracted PGN exists)
    pgn_filename = filename.replace(".zip", ".pgn").replace(".zst", "")
    extracted_path = os.path.join(LOCAL_DIR, pgn_filename)

    if os.path.exists(extracted_path):
        print(f"✅ Already extracted: {pgn_filename}")
        return

    # B. Check if zip/zst exists locally or in Drive
    if not os.path.exists(local_path):
        if os.path.exists(drive_path):
            print(f"📦 Copying {filename} from Drive...")
            shutil.copy(drive_path, local_path)
        else:
            print(f"⬇️ Downloading {filename}...")
            try:
                with requests.get(url, stream=True) as r:
                    r.raise_for_status()
                    with open(local_path, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                # Backup to Drive
                shutil.copy(local_path, drive_path)
            except Exception as e:
                print(f"❌ Failed to download {filename}: {e}")
                return

    # C. Extract Files
    print(f"🔧 Extracting {filename}...")
    try:
        if filename.endswith(".zip"):
            with zipfile.ZipFile(local_path, 'r') as zip_ref:
                zip_ref.extractall(LOCAL_DIR)
            # Zip ফাইলটি কাজ শেষে ডিলিট করে স্পেস বাঁচানো যায় (অপশনাল)
            # os.remove(local_path)
            print(f"   -> Extracted to .pgn")

        elif filename.endswith(".pgn.zst"):
            # ZST আমরা সাধারণত প্রসেসিংয়ের সময় ডিকম্প্রেস করি, তাই এখানে রেখে দিচ্ছি
            print(f"   -> Ready for ZST stream processing")

    except Exception as e:
        print(f"❌ Extraction failed: {e}")

# Run the pipeline
for url in DOWNLOAD_URLS:
    download_and_extract(url)

print("\n✅ All downloads & extractions complete!")

🎯 Targets: 4 files
📂 Local Dir: /content/data
💾 Drive Backup: /content/drive/MyDrive/GambitFlow_Project/Synapse_Data/2/raw_pgns

✅ All downloads & extractions complete!


In [ ]:
# ==============================================================================
# 🏗️ DATABASE STRUCTURE & HELPER FUNCTIONS
# ==============================================================================

import sqlite3
import json
import chess # Import chess library here to avoid NameError

# Database schema
def create_database(db_path):
    """Create opening theory database with proper schema"""
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS openings (
            fen TEXT PRIMARY KEY,
            move_data TEXT,
            total_games INTEGER,
            avg_elo INTEGER
        )
    ''')

    # Performance optimizations for SQLite
    cursor.execute('PRAGMA synchronous = OFF')
    cursor.execute('PRAGMA journal_mode = MEMORY')
    cursor.execute('PRAGMA cache_size = 10000')

    conn.commit()
    return conn

# FEN canonicalization
def canonical_fen(fen):
    """
    Normalize FEN for consistent lookups
    Only keep: position, turn, castling, en passant
    Remove: halfmove clock, fullmove number
    """
    parts = fen.split()
    if len(parts) >= 4:
        return " ".join(parts[:4])
    return fen

# Move quality calculation (PERSPECTIVE-AWARE)
def calculate_move_score(board, result):
    """
    Calculate move quality from current player's perspective.
    If it's White's turn and White won, score is 1.0.
    If it's Black's turn and Black won, score is 1.0.
    """
    is_white_turn = board.turn == chess.WHITE

    if result == '1-0':  # White won
        return 1.0 if is_white_turn else 0.0
    elif result == '0-1':  # Black won
        return 0.0 if is_white_turn else 1.0
    else:  # Draw
        return 0.5

# --- Testing the logic ---
print("🧪 Testing helper functions...")

# Case 1: Start Position (White to move)
test_board = chess.Board()
print(f"Canonical FEN: {canonical_fen(test_board.fen())}")
score_white = calculate_move_score(test_board, '1-0')
print(f"Start Pos (White Turn), Result 1-0 -> Score: {score_white} (Expected: 1.0)")

# Case 2: After 1. e4 (Black to move)
board_after_e4 = chess.Board()
board_after_e4.push_san("e4") # Push move first
score_black = calculate_move_score(board_after_e4, '0-1')
print(f"After e4 (Black Turn), Result 0-1 -> Score: {score_black} (Expected: 1.0)")

# Case 3: After 1. e4 (Black to move), but White won eventually
score_black_loss = calculate_move_score(board_after_e4, '1-0')
print(f"After e4 (Black Turn), Result 1-0 -> Score: {score_black_loss} (Expected: 0.0)")

print("✅ Helpers ready & Logic Verified")

🧪 Testing helper functions...
Canonical FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -
Start Pos (White Turn), Result 1-0 -> Score: 1.0 (Expected: 1.0)
After e4 (Black Turn), Result 0-1 -> Score: 1.0 (Expected: 1.0)
After e4 (Black Turn), Result 1-0 -> Score: 0.0 (Expected: 0.0)
✅ Helpers ready & Logic Verified


In [ ]:
# ==============================================================================
# ⚙️ MAIN PROCESSING: Extract Opening Theory (SYNAPSE-BASE OPTIMIZED)
# ==============================================================================

import os
import time
import json
import sqlite3
import chess.pgn
import zstandard as zstd
import io
import shutil
import traceback

# --- Configuration ---
MIN_ELO = 2400          # Top-level games only
MIN_FREQUENCY = 5       # Set low for distributed processing (Final cleanup will use 50)
MAX_DEPTH = 25          # Process first 25 moves
MAX_BUFFER_SIZE = 30000 # Prevent memory overflow
CHECKPOINT_FILE = os.path.join(DATA_FACTORY, "opening_checkpoint_2.json")

# Database setup
DB_NAME = "opening_theory_2.db"
LOCAL_DB = os.path.join(LOCAL_DIR, DB_NAME)
DRIVE_DB = os.path.join(DATA_FACTORY, DB_NAME)

print(f"🗄️ Database: {LOCAL_DB}")

# Create/load database (copy from Drive if exists)
if os.path.exists(DRIVE_DB) and not os.path.exists(LOCAL_DB):
    print("📥 Copying existing database from Drive...")
    shutil.copy(DRIVE_DB, LOCAL_DB)

# Ensure database exists with correct schema
def create_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS openings (
            fen TEXT PRIMARY KEY,
            move_data TEXT,
            total_games INTEGER,
            avg_elo INTEGER
        )
    ''')
    cursor.execute('PRAGMA synchronous = OFF')
    cursor.execute('PRAGMA journal_mode = MEMORY')
    conn.commit()
    return conn

conn = create_database(LOCAL_DB)
cursor = conn.cursor()

# Checkpoint management
def load_checkpoint():
    if os.path.exists(CHECKPOINT_FILE):
        with open(CHECKPOINT_FILE, 'r') as f:
            return json.load(f)
    return {"games_processed": 0, "positions_found": 0}

def save_checkpoint(games, positions):
    with open(CHECKPOINT_FILE, 'w') as f:
        json.dump({
            "games_processed": games,
            "positions_found": positions,
            "timestamp": time.time()
        }, f)

# Load state
state = load_checkpoint()
start_game = state["games_processed"]
total_positions = state["positions_found"]

print(f"🔄 Resuming from game #{start_game:,}")
print(f"📊 Positions collected so far: {total_positions:,}")

# In-memory buffer
position_buffer = {}

def flush_buffer_to_db():
    """Flush buffer to database, applying distributed frequency filter"""
    global total_positions

    for fen, data in position_buffer.items():
        filtered_moves = {}

        for move_san, move_data in data["moves"].items():
            freq = move_data["frequency"]

            if freq >= MIN_FREQUENCY:
                move_data["avg_score"] = move_data["score_sum"] / freq
                filtered_moves[move_san] = move_data

        if not filtered_moves:
            continue

        avg_elo = data["elo_sum"] // data["total"] if data["total"] > 0 else 0

        cursor.execute('SELECT move_data, total_games, avg_elo FROM openings WHERE fen = ?', (fen,))
        row = cursor.fetchone()

        if row:
            existing_moves = json.loads(row[0])
            existing_total = row[1]
            existing_avg_elo = row[2]

            for move_san, move_data in filtered_moves.items():
                if move_san not in existing_moves:
                    existing_moves[move_san] = {"frequency": 0, "score_sum": 0.0}

                existing_moves[move_san]["frequency"] += move_data["frequency"]
                existing_moves[move_san]["score_sum"] += move_data["score_sum"]
                existing_moves[move_san]["avg_score"] = (
                    existing_moves[move_san]["score_sum"] / existing_moves[move_san]["frequency"]
                )

            new_total = existing_total + data["total"]
            new_avg_elo = ((existing_avg_elo * existing_total) + data["elo_sum"]) // new_total

            cursor.execute(
                'UPDATE openings SET move_data = ?, total_games = ?, avg_elo = ? WHERE fen = ?',
                (json.dumps(existing_moves), new_total, new_avg_elo, fen)
            )
        else:
            cursor.execute(
                'INSERT INTO openings VALUES (?, ?, ?, ?)',
                (fen, json.dumps(filtered_moves), data["total"], avg_elo)
            )
            total_positions += 1

    conn.commit()
    position_buffer.clear()

# Main processing
try:
    # Logic to handle PGN files from Cell 2 extraction
    pgn_files = [f for f in os.listdir(LOCAL_DIR) if f.endswith(".pgn")]

    current_game = 0

    for pgn_file in pgn_files:
        pgn_path = os.path.join(LOCAL_DIR, pgn_file)
        print(f"\n🚀 Processing File: {pgn_file}")

        with open(pgn_path, 'r', encoding='utf-8') as pgn:
            while True:
                game = chess.pgn.read_game(pgn)
                if game is None: break

                current_game += 1
                if current_game <= start_game:
                    if current_game % 100000 == 0: print(f"   Skipped {current_game:,}...")
                    continue

                try:
                    w_elo = int(game.headers.get("WhiteElo", 0))
                    b_elo = int(game.headers.get("BlackElo", 0))
                    if w_elo < MIN_ELO or b_elo < MIN_ELO: continue
                except: continue

                board = game.board()
                result = game.headers.get("Result", "*")
                if result not in ['1-0', '0-1', '1/2-1/2']: continue

                avg_elo = (w_elo + b_elo) // 2

                for move_num, move in enumerate(game.mainline_moves()):
                    if move_num >= MAX_DEPTH: break

                    fen = canonical_fen(board.fen())
                    move_san = board.san(move)
                    score = calculate_move_score(board, result)

                    if fen not in position_buffer:
                        position_buffer[fen] = {"moves": {}, "total": 0, "elo_sum": 0}
                    if move_san not in position_buffer[fen]["moves"]:
                        position_buffer[fen]["moves"][move_san] = {"frequency": 0, "score_sum": 0.0}

                    position_buffer[fen]["moves"][move_san]["frequency"] += 1
                    position_buffer[fen]["moves"][move_san]["score_sum"] += score
                    position_buffer[fen]["total"] += 1
                    position_buffer[fen]["elo_sum"] += avg_elo

                    board.push(move)

                if current_game % 5000 == 0 or len(position_buffer) > MAX_BUFFER_SIZE:
                    flush_buffer_to_db()
                    cursor.execute('SELECT COUNT(*) FROM openings')
                    total_positions = cursor.fetchone()[0]
                    save_checkpoint(current_game, total_positions)
                    try:
                        shutil.copy(LOCAL_DB, DRIVE_DB)
                        backup_status = "✅"
                    except: backup_status = "⚠️ Backup failed"
                    print(f"📌 Game {current_game:,} | Positions: {total_positions:,} | {backup_status}")

except KeyboardInterrupt:
    print("\n⏸️ Interrupted by user")
except Exception as e:
    print(f"\n❌ Error: {e}")
    traceback.print_exc()
finally:
    print("\n💾 Finalizing...")
    if position_buffer: flush_buffer_to_db()
    cursor.execute('SELECT COUNT(*) FROM openings')
    total_positions = cursor.fetchone()[0]
    conn.close()
    save_checkpoint(current_game, total_positions)
    if os.path.exists(LOCAL_DB): shutil.copy(LOCAL_DB, DRIVE_DB)
    print(f"\n✅ Done. Processed: {current_game:,} | Unique: {total_positions:,}")

🗄️ Database: /content/data/opening_theory_2.db
🔄 Resuming from game #0
📊 Positions collected so far: 0

🚀 Processing File: lichess_elite_2025-04.pgn

🚀 Processing File: lichess_elite_2025-05.pgn

🚀 Processing File: lichess_elite_2025-07.pgn

🚀 Processing File: lichess_elite_2025-06.pgn

💾 Finalizing...

✅ Done. Processed: 1,166,986 | Unique: 6,747


In [ ]:
# ==============================================================================
# 📊 VALIDATION & STATISTICS
# ==============================================================================

import sqlite3
import json

# Connect to database
conn = sqlite3.connect(DRIVE_DB)
cursor = conn.cursor()

print("🔍 Analyzing opening database...")
print("=" * 60)

# Basic stats
cursor.execute('SELECT COUNT(*) FROM openings')
total_positions = cursor.fetchone()[0]

cursor.execute('SELECT SUM(total_games) FROM openings')
total_game_moves = cursor.fetchone()[0]

cursor.execute('SELECT AVG(avg_elo) FROM openings')
avg_elo = cursor.fetchone()[0]

print(f"📊 Total unique positions: {total_positions:,}")
print(f"📊 Total game-moves: {total_game_moves:,}")
print(f"📊 Average ELO: {avg_elo:.0f}")

# Sample positions
print("\n🎯 Sample opening positions:")
print("-" * 60)

cursor.execute('SELECT fen, move_data, total_games FROM openings ORDER BY total_games DESC LIMIT 5')

for idx, (fen, move_data_json, total) in enumerate(cursor.fetchall(), 1):
    board = chess.Board(fen)
    moves = json.loads(move_data_json)

    print(f"\n{idx}. Position (seen {total} times):")
    print(f"   FEN: {fen}")

    # Sort moves by frequency
    sorted_moves = sorted(moves.items(), key=lambda x: x[1]["frequency"], reverse=True)

    print(f"   Top moves:")
    for move_san, data in sorted_moves[:3]:
        freq = data["frequency"]
        score = data.get("avg_score", 0.5)
        print(f"   - {move_san}: {freq} games, score {score:.3f}")

# Check data quality
print("\n🔎 Quality check:")
print("-" * 60)

cursor.execute('SELECT fen, move_data FROM openings WHERE total_games >= 100 LIMIT 10')
valid_samples = 0

for fen, move_data_json in cursor.fetchall():
    try:
        board = chess.Board(fen)
        moves = json.loads(move_data_json)

        # Verify moves are legal
        for move_san in moves.keys():
            try:
                board.parse_san(move_san)
                valid_samples += 1
            except:
                print(f"⚠️ Invalid move {move_san} in position {fen}")
                break
    except Exception as e:
        print(f"⚠️ Error processing {fen}: {e}")

print(f"✅ Validated {valid_samples} position-move pairs")

# Database size
db_size_mb = os.path.getsize(DRIVE_DB) / (1024**2)
print(f"\n💾 Database size: {db_size_mb:.2f} MB")

conn.close()

print("\n" + "=" * 60)
print("✅ Opening database v2 is ready!")
print(f"📂 Location: {DRIVE_DB}")
print("\n🎉 Next step: Upload to Hugging Face")

🔍 Analyzing opening database...
📊 Total unique positions: 6,747
📊 Total game-moves: 7,070,865
📊 Average ELO: 2681

🎯 Sample opening positions:
------------------------------------------------------------

1. Position (seen 1022854 times):
   FEN: rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq -
   Top moves:
   - e4: 475754 games, score 0.527
   - d4: 319756 games, score 0.524
   - Nf3: 108411 games, score 0.533

2. Position (seen 475613 times):
   FEN: rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq -
   Top moves:
   - c5: 168950 games, score 0.462
   - e5: 127260 games, score 0.482
   - c6: 53282 games, score 0.468

3. Position (seen 319681 times):
   FEN: rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq -
   Top moves:
   - Nf6: 169190 games, score 0.477
   - d5: 89092 games, score 0.469
   - e6: 13326 games, score 0.478

4. Position (seen 169142 times):
   FEN: rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBNR w KQkq -
   Top moves:
   - c4: 96822 games, score 0.527
   

In [ ]:
# ==============================================================================
# 🧠 GAMBITFLOW - SUPER MERGER SCRIPT
# ==============================================================================

import sqlite3
import json
import os
import time

# --- 1. Configuration ---
# আপনার ৩টি সোর্স ডেটাবেসের পাথ
SOURCE_DBS = [
    "/content/drive/MyDrive/GambitFlow_Project/Synapse_Data/1/opening_theory_1.db",
    "/content/drive/MyDrive/GambitFlow_Project/Synapse_Data/2/opening_theory_2.db",
    "/content/drive/MyDrive/GambitFlow_Project/Synapse_Data/opening_theory_v2_clean.db"
]

# নতুন মাস্টার ডেটাবেসের পাথ
MASTER_DB = "/content/drive/MyDrive/GambitFlow_Project/Synapse_Data/opening_theory_master.db"

print("🚀 Starting Database Merge Process...")
print(f"Master DB will be created at: {MASTER_DB}")
print("-" * 60)

# --- 2. Setup ---
# যদি আগের কোনো মাস্টার ফাইল থাকে, তবে ডিলিট করা হবে
if os.path.exists(MASTER_DB):
    os.remove(MASTER_DB)
    print("🗑️ Removed old master database to start fresh.")

# নতুন মাস্টার ডেটাবেস তৈরি
master_conn = sqlite3.connect(MASTER_DB)
master_cursor = master_conn.cursor()
master_cursor.execute('''
    CREATE TABLE openings (
        fen TEXT PRIMARY KEY,
        move_data TEXT,
        total_games INTEGER,
        avg_elo INTEGER
    )
''')
master_cursor.execute('PRAGMA synchronous = OFF')
master_cursor.execute('PRAGMA journal_mode = MEMORY')

# --- 3. Core Merge Logic ---
# আমরা র‍্যামে একটি ডিকশনারি ব্যবহার করব যা অনেক ফাস্ট
master_data = {}
total_positions_processed = 0

print("🧠 Reading and merging data in memory...")

for db_path in SOURCE_DBS:
    print(f"   -> Processing: {os.path.basename(db_path)}")
    if not os.path.exists(db_path):
        print(f"   ⚠️ WARNING: File not found, skipping: {db_path}")
        continue

    src_conn = sqlite3.connect(db_path)
    src_cursor = src_conn.cursor()
    src_cursor.execute("SELECT fen, move_data, total_games, avg_elo FROM openings")

    while True:
        rows = src_cursor.fetchmany(5000)
        if not rows:
            break

        for fen, move_data_json, total_games, avg_elo in rows:
            total_positions_processed += 1
            new_moves = json.loads(move_data_json)

            if fen in master_data:
                # --- MERGE LOGIC ---
                existing = master_data[fen]

                # Update weighted ELO
                new_total_games = existing['total_games'] + total_games
                existing['avg_elo'] = (
                    (existing['avg_elo'] * existing['total_games']) + (avg_elo * total_games)
                ) // new_total_games
                existing['total_games'] = new_total_games

                # Merge moves
                for move, data in new_moves.items():
                    if move in existing['moves']:
                        # If move exists, sum stats
                        existing['moves'][move]['frequency'] += data.get('frequency', 0)
                        existing['moves'][move]['score_sum'] += data.get('score_sum', 0)
                    else:
                        # If new move, add it
                        existing['moves'][move] = data
            else:
                # --- NEW FEN ---
                master_data[fen] = {
                    'moves': new_moves,
                    'total_games': total_games,
                    'avg_elo': avg_elo
                }

    src_conn.close()

print("\n💾 Writing merged data to new database...")

# --- 4. Write to Master DB ---
batch_data = []
for fen, data in master_data.items():
    batch_data.append((
        fen,
        json.dumps(data['moves']),
        data['total_games'],
        data['avg_elo']
    ))

master_cursor.executemany("INSERT INTO openings VALUES (?, ?, ?, ?)", batch_data)
master_conn.commit()
master_conn.close()

# --- 5. Final Statistics ---
final_size_mb = os.path.getsize(MASTER_DB) / (1024**2)

print("-" * 60)
print("✅ MERGE COMPLETE!")
print(f"📊 Total source positions processed: {total_positions_processed:,}")
print(f"✨ Final unique positions in Master DB: {len(master_data):,}")
print(f"💾 Final Master DB Size: {final_size_mb:.2f} MB")
print("\n🎉 Your powerful, merged opening database is ready!")

🚀 Starting Database Merge Process...
Master DB will be created at: /content/drive/MyDrive/GambitFlow_Project/Synapse_Data/opening_theory_master.db
------------------------------------------------------------
🧠 Reading and merging data in memory...
   -> Processing: opening_theory_1.db
   -> Processing: opening_theory_2.db
   -> Processing: opening_theory_v2_clean.db

💾 Writing merged data to new database...
------------------------------------------------------------
✅ MERGE COMPLETE!
📊 Total source positions processed: 19,157
✨ Final unique positions in Master DB: 11,054
💾 Final Master DB Size: 2.62 MB

🎉 Your powerful, merged opening database is ready!


In [ ]:
# ==============================================================================
# 🚀 UPLOAD MASTER DATABASE TO HUGGING FACE
# ==============================================================================

from huggingface_hub import HfApi
import os

# --- 1. Configuration ---
# ⚠️ আপনার Hugging Face Write Token টি এখানে দিন
HF_TOKEN = "HF"

HF_USERNAME = "GambitFlow"
REPO_ID = f"{HF_USERNAME}/Opening-Database"
SOURCE_FILE = "/content/drive/MyDrive/GambitFlow_Project/Synapse_Data/opening_theory_master.db"
TARGET_FILENAME = "opening_theory.db" # HF রিপোজিটরিতে এই নামে সেভ হবে

print("🚀 Initializing Upload to Hugging Face...")
print(f"📂 Source: {os.path.basename(SOURCE_FILE)}")
print(f"☁️ Target Repo: {REPO_ID}")
print("-" * 50)

if not os.path.exists(SOURCE_FILE):
    print("❌ FATAL ERROR: Master database file not found!")
else:
    file_size_mb = os.path.getsize(SOURCE_FILE) / (1024**2)
    print(f"⚖️ Upload Size: {file_size_mb:.2f} MB")

    try:
        api = HfApi(token=HF_TOKEN)

        # ফাইল আপলোড করা হচ্ছে
        print("\n⏳ Uploading... (Please wait)")
        api.upload_file(
            path_or_fileobj=SOURCE_FILE,
            path_in_repo=TARGET_FILENAME,
            repo_id=REPO_ID,
            repo_type="dataset",
            commit_message="Upload master opening theory (merged from 3 sources)"
        )

        print("\n✅ UPLOAD SUCCESSFUL!")
        print(f"🔗 Dataset URL: https://huggingface.co/datasets/{REPO_ID}")
        print(f"📥 Direct Link: https://huggingface.co/datasets/{REPO_ID}/resolve/main/{TARGET_FILENAME}")

    except Exception as e:
        print(f"\n❌ Upload Failed: {e}")
        print("👉 Tip: Check if your HF_TOKEN has 'Write' permissions or is correct.")

🚀 Initializing Upload to Hugging Face...
📂 Source: opening_theory_master.db
☁️ Target Repo: GambitFlow/Opening-Database
--------------------------------------------------
⚖️ Upload Size: 2.62 MB

⏳ Uploading... (Please wait)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  .../opening_theory_master.db:  57%|#####7    | 1.57MB / 2.75MB            


✅ UPLOAD SUCCESSFUL!
🔗 Dataset URL: https://huggingface.co/datasets/GambitFlow/Opening-Database
📥 Direct Link: https://huggingface.co/datasets/GambitFlow/Opening-Database/resolve/main/opening_theory.db
